In [1]:
print sc

In [48]:
from pyspark.sql import SQLContext
import requests, StringIO, pandas as pd, json, re,numpy as np,os
from pyspark.mllib.feature import PCA as PCAmllib
import pyspark.mllib.linalg as linalg
from pyspark.mllib.feature import PCA,PCAModel,StandardScaler
from pyspark.mllib.linalg.distributed import RowMatrix
from pyspark.mllib.stat import MultivariateStatisticalSummary
from pyspark.mllib.stat import Statistics
from pyspark.mllib.classification import LogisticRegressionWithSGD
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.evaluation import BinaryClassificationMetrics

In [39]:
sqlContext=SQLContext(sc)
print sqlContext

In [3]:
credentials_1 = {
  'auth_uri':'',
  'global_account_auth_uri':'',
  'username':'admin_56b4be3a96710cbeade2022afc6cc5b2420caafd',
  'password':"""VK!/x8a.2S*q).tR""",
  'auth_url':'https://identity.open.softlayer.com',
  'project':'object_storage_0a2e2492_b9f0_4ca3_a7a7_a34470fe9463',
  'project_id':'5aef2dbbd530481b9ca5dcea9d7ed808',
  'region':'dallas',
  'user_id':'1774093a5b944eacbd8f1aee39fbc163',
  'domain_id':'6d20c55908574220acf681abf2b946ca',
  'domain_name':'1072767',
  'filename':'spambase_data.csv',
  'container':'Spark_Learning',
  'tenantId':'s3bc-3ce8b2b028f00f-43612ebad61e'
}

In [4]:
def get_file_content(credentials):
    """For given credentials, this functions returns a StringIO object containing the file content."""

    url1 = ''.join([credentials['auth_url'], '/v3/auth/tokens'])
    data = {'auth': {'identity': {'methods': ['password'],
            'password': {'user': {'name': credentials['username'],'domain': {'id': credentials['domain_id']},
            'password': credentials['password']}}}}}
    headers1 = {'Content-Type': 'application/json'}
    resp1 = requests.post(url=url1, data=json.dumps(data), headers=headers1)
    resp1_body = resp1.json()
    for e1 in resp1_body['token']['catalog']:
        if(e1['type']=='object-store'):
            for e2 in e1['endpoints']:
                if(e2['interface']=='public'and e2['region']==credentials['region']):
                    url2 = ''.join([e2['url'],'/', credentials['container'], '/', credentials['filename']])
    s_subject_token = resp1.headers['x-subject-token']
    headers2 = {'X-Auth-Token': s_subject_token, 'accept': 'application/json'}
    resp2 = requests.get(url=url2, headers=headers2)
    return StringIO.StringIO(resp2.content)

In [5]:
content_string = get_file_content(credentials_1)
spam_database = pd.read_csv(content_string)

In [6]:
spam_database.head()

word_freq_make  word_freq_address  word_freq_all  word_freq_3d  \
0            0.00               0.64           0.64             0   
1            0.21               0.28           0.50             0   
2            0.06               0.00           0.71             0   
3            0.00               0.00           0.00             0   
4            0.00               0.00           0.00             0   

   word_freq_our  word_freq_over  word_freq_remove  word_freq_internet  \
0           0.32            0.00              0.00                0.00   
1           0.14            0.28              0.21                0.07   
2           1.23            0.19              0.19                0.12   
3           0.63            0.00              0.31                0.63   
4           0.63            0.00              0.31                0.63   

   word_freq_order  word_freq_mail    ...     char_freq_;  char_freq_(  \
0             0.00            0.00    ...            0.00        0.000   
1             0.00            0.94    ...            0.00        0.132   
2             0.64            0.25    ...            0.01        0.143   
3             0.31            0.63    ...            0.00        0.137   
4             0.31            0.63    ...            0.00        0.135   

   char_freq_[  char_freq_!  char_freq_$  char_freq_#  \
0            0        0.778        0.000        0.000   
1            0        0.372        0.180        0.048   
2            0        0.276        0.184        0.010   
3            0        0.137        0.000        0.000   
4            0        0.135        0.000        0.000   

   capital_run_length_average  capital_run_length_longest  \
0                       3.756                          61   
1                       5.114                         101   
2                       9.821                         485   
3                       3.537                          40   
4                       3.537                          40   

   capital_run_length_total  spam_flg  
0                       278         1  
1                      1028         1  
2                      2259         1  
3                       191         1  
4                       191         1  

[5 rows x 58 columns]

In [7]:
os.getcwd()

'/gpfs/global_fs01/sym_shared/YPProdSpark/user/s3bc-3ce8b2b028f00f-43612ebad61e/notebook/work'

In [8]:
spam_database.to_csv('spam_database.csv',index=False)

THIS IS WHERE WE START IN A NORMAL DEVELOPMENT CYCLE

In [9]:
spam_rdd=sc.textFile('spam_database.csv')
header = spam_rdd.first() #extract header
spam_rdd = spam_rdd.filter(lambda x:x !=header)
spam_rdd=spam_rdd.map(lambda x:x.split(',')).map(lambda x:np.asarray(x))

In [10]:
spam_rdd.take(2)

[array([u'0.0', u'0.64', u'0.64', u'0.0', u'0.32', u'0.0', u'0.0', u'0.0',
        u'0.0', u'0.0', u'0.0', u'0.64', u'0.0', u'0.0', u'0.0', u'0.32',
        u'0.0', u'1.29', u'1.93', u'0.0', u'0.96', u'0.0', u'0.0', u'0.0',
        u'0.0', u'0.0', u'0.0', u'0.0', u'0.0', u'0.0', u'0.0', u'0.0',
        u'0.0', u'0.0', u'0.0', u'0.0', u'0.0', u'0.0', u'0.0', u'0.0',
        u'0.0', u'0.0', u'0.0', u'0.0', u'0.0', u'0.0', u'0.0', u'0.0',
        u'0.0', u'0.0', u'0.0', u'0.778', u'0.0', u'0.0', u'3.756', u'61',
        u'278', u'1'], 
       dtype='<U5'),
 array([u'0.21', u'0.28', u'0.5', u'0.0', u'0.14', u'0.28', u'0.21',
        u'0.07', u'0.0', u'0.94', u'0.21', u'0.79', u'0.65', u'0.21',
        u'0.14', u'0.14', u'0.07', u'0.28', u'3.47', u'0.0', u'1.59',
        u'0.0', u'0.43', u'0.43', u'0.0', u'0.0', u'0.0', u'0.0', u'0.0',
        u'0.0', u'0.0', u'0.0', u'0.0', u'0.0', u'0.0', u'0.0', u'0.07',
        u'0.0', u'0.0', u'0.0', u'0.0', u'0.0', u'0.0', u'0.0', u'0.0',
        u'0.

We lock down the OOS data

In [11]:
spam_rdd_splits=spam_rdd.randomSplit([0.8,0.2])

In [12]:
spam_rdd_cv=spam_rdd_splits[0]
spam_rdd_oos=spam_rdd_splits[1]

Now Let us work with the Cross-validation dataset only

In [13]:
spam_rdd_features=spam_rdd_cv.map(lambda x:x[:-1])
spam_rdd_response=spam_rdd_cv.map(lambda x:x[-1])

In [14]:
print spam_rdd_features.take(1)
print spam_rdd_response.take(1)

[array([u'0.0', u'0.64', u'0.64', u'0.0', u'0.32', u'0.0', u'0.0', u'0.0',
       u'0.0', u'0.0', u'0.0', u'0.64', u'0.0', u'0.0', u'0.0', u'0.32',
       u'0.0', u'1.29', u'1.93', u'0.0', u'0.96', u'0.0', u'0.0', u'0.0',
       u'0.0', u'0.0', u'0.0', u'0.0', u'0.0', u'0.0', u'0.0', u'0.0',
       u'0.0', u'0.0', u'0.0', u'0.0', u'0.0', u'0.0', u'0.0', u'0.0',
       u'0.0', u'0.0', u'0.0', u'0.0', u'0.0', u'0.0', u'0.0', u'0.0',
       u'0.0', u'0.0', u'0.0', u'0.778', u'0.0', u'0.0', u'3.756', u'61',
       u'278'], 
      dtype='<U5')]
[u'1']


In [15]:
standardizer = StandardScaler(True,True)
model=standardizer.fit(spam_rdd_features)
standard_spam_feature=model.transform(spam_rdd_features)

In [16]:
cStats = Statistics.colStats(spam_rdd_features)

In [17]:
print cStats.mean()
print cStats.variance()

[  1.04863918e-01   2.27167879e-01   2.78881703e-01   7.20991646e-02
   3.10231743e-01   9.45001347e-02   1.12969550e-01   1.03654002e-01
   8.91673403e-02   2.38353544e-01   5.97682565e-02   5.44645648e-01
   9.39396389e-02   5.86364861e-02   4.71732687e-02   2.52414444e-01
   1.41180275e-01   1.88798168e-01   1.65286176e+00   8.31177580e-02
   8.19832929e-01   1.25944489e-01   9.92508758e-02   9.46752897e-02
   5.50935058e-01   2.67836163e-01   7.71538669e-01   1.16389113e-01
   9.73672864e-02   9.94987874e-02   6.50687146e-02   4.66451091e-02
   1.01110213e-01   4.76825653e-02   9.98248451e-02   9.64699542e-02
   1.35249259e-01   1.05658852e-02   7.74562113e-02   6.40635947e-02
   4.48773915e-02   1.36545406e-01   4.66289410e-02   8.21180275e-02
   2.87855026e-01   1.78892482e-01   5.20075451e-03   3.14389652e-02
   4.20404204e-02   1.39551873e-01   1.63031528e-02   2.62062517e-01
   7.38218809e-02   4.10145513e-02   5.37067367e+00   5.18873619e+01
   2.80272703e+02]
[  9.67804599e-

In [18]:
numIters = 100
stepSize = 10.
regParam = 1e-6
regType = 'l2'
includeIntercept = True

In [19]:
#trainData=spam_rdd_cv.map(lambda line:LabeledPoint(line.split(' ')[-1], line.split(' ')[1:]))
trainData=spam_rdd_cv.map(lambda x:LabeledPoint(x[-1], x[:-1]))

In [20]:
trainData.take(3)

[LabeledPoint(1.0, [0.0,0.64,0.64,0.0,0.32,0.0,0.0,0.0,0.0,0.0,0.0,0.64,0.0,0.0,0.0,0.32,0.0,1.29,1.93,0.0,0.96,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.778,0.0,0.0,3.756,61.0,278.0]),
 LabeledPoint(1.0, [0.21,0.28,0.5,0.0,0.14,0.28,0.21,0.07,0.0,0.94,0.21,0.79,0.65,0.21,0.14,0.14,0.07,0.28,3.47,0.0,1.59,0.0,0.43,0.43,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.07,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.132,0.0,0.372,0.18,0.048,5.114,101.0,1028.0]),
 LabeledPoint(1.0, [0.06,0.0,0.71,0.0,1.23,0.19,0.19,0.12,0.64,0.25,0.38,0.45,0.12,0.0,1.75,0.06,0.06,1.03,1.36,0.32,0.51,0.0,1.16,0.06,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.06,0.0,0.0,0.12,0.0,0.06,0.06,0.0,0.0,0.01,0.143,0.0,0.276,0.184,0.01,9.821,485.0,2259.0])]

In [47]:
print trainData.map(lambda x:x.label).take(2)
print trainData.map(lambda x:x.features).take(2)

[1.0, 1.0]
[DenseVector([0.0, 0.64, 0.64, 0.0, 0.32, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.64, 0.0, 0.0, 0.0, 0.32, 0.0, 1.29, 1.93, 0.0, 0.96, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.778, 0.0, 0.0, 3.756, 61.0, 278.0]), DenseVector([0.21, 0.28, 0.5, 0.0, 0.14, 0.28, 0.21, 0.07, 0.0, 0.94, 0.21, 0.79, 0.65, 0.21, 0.14, 0.14, 0.07, 0.28, 3.47, 0.0, 1.59, 0.0, 0.43, 0.43, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.07, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.132, 0.0, 0.372, 0.18, 0.048, 5.114, 101.0, 1028.0])]


In [21]:
model0 = LogisticRegressionWithSGD.train(trainData, iterations = numIters, step = stepSize, regParam = regParam, regType = regType, intercept = includeIntercept)
print model0.weights, model0.intercept

[-2.67785181075,-19.5176533014,-6.60921082004,3.13139741211,-2.53371183764,0.548833608402,4.53393188819,1.23908292186,0.831621366731,-6.00172827576,0.496286495181,-29.6316032118,-1.5220181083,-1.46197960287,1.47014823429,5.15551695577,1.92915351555,-0.662878217968,-45.4068560686,3.26613475025,-5.15918380615,1.41356543756,4.80654445164,3.38396985821,-65.8780487881,-32.1566617966,-60.0390534137,-13.8837556651,-12.2253307766,-12.2178128369,-8.43723576353,-5.97943150754,-11.0943843787,-5.98978455765,-12.5184449063,-9.81878323014,-13.9670568756,-0.978263610986,-9.05822107793,-5.72611869067,-5.96256396064,-16.6573187918,-5.21937763736,-9.80665831376,-26.9072599642,-21.4665978757,-0.485913793108,-3.83357355577,-3.55468392047,-9.57519265263,-1.49463211278,1.63204640266,2.9318559101,-0.195332706019,-69.7760567319,145.310602062,-35.3635698717] -51.3598746919


In [23]:
column_names=header.split(',')

In [24]:
model_results={}
model_results['intercept']=model0.intercept
for i in range(0,len(model0.weights)):
    model_results[column_names[i]]=model0.weights[i]
print model_results

{u'capital_run_length_longest': 145.31060206177469, u'word_freq_1999': -13.967056875600607, u'word_freq_original': -5.2193776373621779, u'word_freq_project': -9.8066583137578185, u'word_freq_credit': 3.2661347502466485, u'char_freq_[': -1.4946321127845834, u'word_freq_order': 0.83162136673097276, u'word_freq_remove': 4.5339318881872961, u'word_freq_will': -29.631603211754239, u'char_freq_$': 2.9318559101009933, u'word_freq_receive': 0.4962864951812655, u'word_freq_415': -5.9897845576527375, u'word_freq_over': 0.54883360840174156, u'word_freq_technology': -9.8187832301417615, u'word_freq_money': 3.3839698582065179, u'capital_run_length_average': -69.776056731851924, u'word_freq_parts': -0.97826361098586956, u'word_freq_font': 1.4135654375590856, u'word_freq_george': -60.039053413724474, u'word_freq_all': -6.6092108200429456, u'capital_run_length_total': -35.363569871704215, u'word_freq_people': -1.5220181083032611, u'word_freq_table': -0.48591379310815536, u'word_freq_cs': -5.9625639606

In [26]:
cross_validation_size=[0.1]*10
print cross_validation_size

[0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]


In [27]:
cv_splits=spam_rdd_cv.randomSplit(cross_validation_size)

In [28]:
enumerate(cv_splits)

In [33]:
fold_list=[x for x in range(0,len(cross_validation_size))]
for num_fold in range(0,len(cross_validation_size)):
    indexes=[elt for num, elt in enumerate(fold_list) if not num == num_fold]
    print num_fold, indexes

0 [1, 2, 3, 4, 5, 6, 7, 8, 9]
1 [0, 2, 3, 4, 5, 6, 7, 8, 9]
2 [0, 1, 3, 4, 5, 6, 7, 8, 9]
3 [0, 1, 2, 4, 5, 6, 7, 8, 9]
4 [0, 1, 2, 3, 5, 6, 7, 8, 9]
5 [0, 1, 2, 3, 4, 6, 7, 8, 9]
6 [0, 1, 2, 3, 4, 5, 7, 8, 9]
7 [0, 1, 2, 3, 4, 5, 6, 8, 9]
8 [0, 1, 2, 3, 4, 5, 6, 7, 9]
9 [0, 1, 2, 3, 4, 5, 6, 7, 8]


In [ ]:
numIters = 1000
stepSize = 10.
regType = 'l2'
includeIntercept = True
#
list_of_models={}
fold_list=[x for x in range(0,len(cross_validation_size))]
for num_fold in range(0,len(cross_validation_size)):
    indexes=[elt for num, elt in enumerate(fold_list) if not num == num_fold]
    first_index=indexes[0]
    train_rdd=cv_splits[first_index]
    for index in indexes:
        train_rdd=train_rdd.union(cv_splits[index])
    val_rdd=cv_splits[num_fold]
    print num_fold,
    print train_rdd.count(),
    print val_rdd.count()
    Cs=list(np.power(10.0, np.arange(-5, 5)))
    trainData=train_rdd.map(lambda x:LabeledPoint(x[-1], x[:-1]))
    valData=val_rdd.map(lambda x:LabeledPoint(x[-1], x[:-1]))    
    for each_C in Cs:
        list_of_models['num_fold']=num_fold
        list_of_models['C']=each_C
        regParam = each_C
        this_model = LogisticRegressionWithSGD.train(trainData, iterations = numIters, step = stepSize, regParam = regParam, regType = regType, intercept = includeIntercept)
        list_of_models['model']=this_model
        predictionAndLabels  = valData.map(lambda p: (this_model.predict(p.features)),this_model)
        list_of_models['metrics']= BinaryClassificationMetrics(predictionAndLabels)       

In [54]:
print Cs

[1.0000000000000001e-05, 0.0001, 0.001, 0.01, 0.10000000000000001, 1.0, 10.0, 100.0, 1000.0, 10000.0]
